In [25]:
from uuid import uuid4
from os import remove
from os.path import join
from tempfile import gettempdir
import numpy as np
import pandas as pd

from aequilibrae.utils.create_example import create_example

# Imports for SF transit graph construction
from aequilibrae.project.database_connection import database_connection

In [2]:
fldr = join(gettempdir(), uuid4().hex)
project = create_example(fldr, "coquimbo")

In [3]:
project.network.build_graphs()
car_graph = project.network.graphs["c"]
car_graph.set_blocked_centroid_flows(False)

bike_graph = project.network.graphs["c"]
bike_graph.set_blocked_centroid_flows(False)

/home/ubuntu/.local/lib/python3.10/site-packages/aequilibrae/project/network/network.py:337: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = pd.read_sql(sql, conn).fillna(value=np.nan)


In [4]:
# Preload parameters
period_start = int(6.5 * 60 * 60) # 6:30am in seconds from midnight
period_end = int(8.5 * 60 * 60)   # 8:30am in seconds from midnight

In [36]:
conn = database_connection("transit")

select_trip_ids = f"SELECT DISTINCT trip_id FROM trips_schedule WHERE arrival BETWEEN {period_start} AND {period_end}"

# Convert trip_id's to corresponding pattern_id's
select_pattern_ids = f"SELECT pattern_id FROM trips WHERE trip_id IN ({select_trip_ids})"

# Convert pattern id's to groups of link/dir's
select_links = f"WITH X AS ({select_pattern_ids}) SELECT pm.link, pm.dir FROM X INNER JOIN pattern_mapping pm ON X.pattern_id = pm.pattern_id"

In [44]:
links = pd.DataFrame(conn.execute(select_links).fetchall(), columns=['link_id', 'direction'])
links['direction'] = links['direction'].apply(lambda d: 1 - 2 * d) # Fixing the 0/1 issue (don't know why it's still there)
links['PCE'] = 1
links = links.groupby(['link_id', 'direction'], as_index=False)['PCE'].sum().rename(columns={'PCE': 'preload'})


In [45]:
links

,link_id,direction,preload
0,2,1,41
1,12,1,46
2,15,1,22
3,16,1,22
4,17,1,44
...,...,...,...
2593,34777,1,15
2594,34778,1,15
2595,34779,1,15
2596,34780,1,30


In [59]:
preload = pd.merge(car_graph.graph, links, on=['link_id', 'direction'], how='left')
preload['preload'] = preload['preload'].fillna(0)
preload = preload.sort_values(by='__supernet_id__')['preload'].to_numpy()


117719.0

In [ ]:
conn.close()